In [52]:
from qiskit.opflow import X, Z, I, SummedOp
from qiskit.quantum_info import Pauli
from qiskit.opflow import PauliSumOp
from qiskit.circuit.library import TwoLocal
import numpy as np

# Function for Jordan-Wigner Mapping

In [53]:
def jordan_wigner(N):
    """
    Jordan-Wigner transformation for N qubits.
    """
    sigma_x = np.array([[0, 1], [1, 0]])
    sigma_z = np.array([[1, 0], [0, -1]])
    X = np.eye(2)
    Z = np.eye(2)
    for i in range(N-1):
        X = np.kron(X, sigma_x)
        Z = np.kron(Z, sigma_z)
    X_list = [np.eye(2**(N-i-1)) for i in range(N)]
    Z_list = [np.eye(2**(N-i-1)) for i in range(N)]
    for i in range(N):
        X_list[i] = np.kron(X_list[i], sigma_x)
        Z_list[i] = np.kron(Z_list[i], sigma_z)
    Y_list = [1j*a.dot(b) for a,b in zip(X_list,Z_list)]
    return X,Y_list,Z

In [54]:
#This function is working well as we can verify using the following example.
# Example usage
N = 4
jw = jordan_wigner(N)
print(jw)

(array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.

# Function to convert Second_Q_Hamiltonian into Q_Hamiltonian

In [55]:
def get_qubit_hamiltonian(N, epsilon, V):
    """
    Convert the second quantized Hamiltonian to qubit Hamiltonian using the
    Jordan-Wigner transformation.
    """
    X,Y,Z = jordan_wigner(N)
    H_one = SummedOp([epsilon[i]*Z[i] for i in range(N)])
    H_two = SummedOp([V[i,j]*(np.eye(2**i) - Z[i])/2*(np.eye(2**j) - Z[j])/2*X[i]*X[j] for i in range(N) for j in range(i+1, N)])
    H_qubit = H_one + H_two
    return H_qubit

In [56]:
#This funtion is not working.Probably the argumests it takes do not satisfy the type of objects ListOp demands.
N = 4
epsilon = np.array([-1.0, 0.0, 0.0, 1.0])
V = np.array([[0.0, 1.0, 0.0, 0.0],
              [1.0, 0.0, 1.0, 0.0],
              [0.0, 1.0, 0.0, 1.0],
              [0.0, 0.0, 1.0, 0.0]])

print(epsilon)
print(V)
H_qubit = get_qubit_hamiltonian(N, epsilon, V)
print(H_qubit) # Print the qubit Hamiltonian as a sum of Pauli operators

[-1.  0.  0.  1.]
[[0. 1. 0. 0.]
 [1. 0. 1. 0.]
 [0. 1. 0. 1.]
 [0. 0. 1. 0.]]


TypeError: ListOp expecting objects of type OperatorBase, got [-1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]

In [90]:
V = np.array([[0.0, 1.0, 0.0, 0.0],
              [1.0, 0.0, 1.0, 0.0],
              [0.0, 1.0, 0.0, 1.0],
              [0.0, 0.0, 1.0, 0.0]])
epsilon = np.array([-1.0, 0.0, 0.0, 1.0])
X,Y,Z = jordan_wigner(N)
sumop = np.zeros((1, 16))
for i in range(N):
    H_one = np.array(epsilon[i]*Z[i])
    sumop = sumop + H_one
#     H_two = SummedOp([V[i,j]*(np.eye(2**i) - Z[i])/2*(np.eye(2**j) - Z[j])/2*X[i]*X[j] for i in range(N) for j in range(i+1, N)])
#     H_qubit = H_one + H_two
#     return H_qubit

for i in range(N):
    for j in range(i+1, N):
#         x = V[i,j]*(np.eye(2**i) - Z[i])/2*(np.eye(2**j) - Z[j])/2*X[i]*X[j]
        x = (np.eye(2**i))
        y = np.array(Z[i])
        x = x.ravel().tolist()
        
        for k in range(len(x)+1, 17):
            x.append(0)
        
        x_j = (np.eye(2**j))
        y_j = np.array(Z[j])
        x_j = x_j.ravel().tolist()
        
#         for k in range(len(x_j)+1, 17):
#             x_j.append(0)

            
#         print("i-> ", V[i,j]*(x - y))
        print("x_j:",len(x_j))
        print("y_j:",len(y_j))

# print(sumop)

x_j: 4
y_j: 16
x_j: 16
y_j: 16
x_j: 64
y_j: 16
x_j: 16
y_j: 16
x_j: 64
y_j: 16
x_j: 64
y_j: 16


In [91]:
V = np.array([[0.0, 1.0, 0.0, 0.0],
              [1.0, 0.0, 1.0, 0.0],
              [0.0, 1.0, 0.0, 1.0],
              [0.0, 0.0, 1.0, 0.0]])
epsilon = np.array([-1.0, 0.0, 0.0, 1.0])
X,Y,Z = jordan_wigner(N)
result = []
for i in range(N):
    for j in range(i+1, N):
        op_i = np.eye(2**i) - Z[i]
        op_j = np.eye(2**j) - Z[j]
        op_ij = X[i] * X[j]
        term = V[i, j] * (op_i / 2) * (op_j / 2) * op_ij
        result.append(term)

ValueError: operands could not be broadcast together with shapes (2,2) (16,) 